# Simple API Example Usage

Author: Ryan Fogle

In [17]:
import os
from cta import SimpleAPI
from dotenv import load_dotenv
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
load_dotenv(override=True)

True

## GET Time

In [2]:
tracker = SimpleAPI(key=os.environ['CTA_KEY'])
tracker.gettime()

{'bustime-response': {'tm': '20240228 22:40:16'}}

## GET Data Feeds

This is not used for CTA, but included for sake of mimicking CTA HTTP routes. 

In [3]:
tracker.getrtpidatafeeds()

{'bustime-response': {'rtpidatafeeds': [{'name': 'bustime',
    'source': 'Bus Tracker',
    'displayname': 'CTA',
    'enabled': 'true',
    'visible': 'true'}]}}

## GET Routes

In [7]:
routes = tracker.getroutes()
len(routes['bustime-response']['routes'])

124

In [16]:
df = pd.DataFrame(routes['bustime-response']['routes'])
df

,rt,rtnm,rtclr,rtdd
0,1,Bronzeville/Union Station,#336633,1
1,2,Hyde Park Express,#993366,2
2,3,King Drive,#009900,3
3,4,Cottage Grove,#cc3300,4
4,X4,Cottage Grove Express,#006666,X4
...,...,...,...,...
119,171,U. of Chicago/Hyde Park,#336633,171
120,172,U. of Chicago/Kenwood,#993366,172
121,192,U. of Chicago Hospitals Express,#cc3300,192
122,201,Central/Ridge,#996633,201


In [18]:
df['directions'] = df['rt'].progress_apply(lambda x: tracker.getdirections(x))
df

  0%|          | 0/124 [00:00<?, ?it/s]

100%|██████████| 124/124 [00:55<00:00,  2.25it/s]


,rt,rtnm,rtclr,rtdd,directions
0,1,Bronzeville/Union Station,#336633,1,{'bustime-response': {'directions': [{'id': 'N...
1,2,Hyde Park Express,#993366,2,{'bustime-response': {'directions': [{'id': 'N...
2,3,King Drive,#009900,3,{'bustime-response': {'directions': [{'id': 'N...
3,4,Cottage Grove,#cc3300,4,{'bustime-response': {'directions': [{'id': 'N...
4,X4,Cottage Grove Express,#006666,X4,{'bustime-response': {'directions': [{'id': 'N...
...,...,...,...,...,...
119,171,U. of Chicago/Hyde Park,#336633,171,{'bustime-response': {'directions': [{'id': 'N...
120,172,U. of Chicago/Kenwood,#993366,172,{'bustime-response': {'directions': [{'id': 'N...
121,192,U. of Chicago Hospitals Express,#cc3300,192,{'bustime-response': {'directions': [{'id': 'N...
122,201,Central/Ridge,#996633,201,{'bustime-response': {'directions': [{'id': 'E...


In [24]:
df['directions_clean'] = df['directions'].apply(lambda x: x['bustime-response']['directions'])
df_dir = df.explode('directions_clean')
df_dir['dir_id'] = df_dir['directions_clean'].apply(lambda x: x['id'])
df_dir['dir_nm'] = df_dir['directions_clean'].apply(lambda x: x['name'])
df2 = df_dir.drop(columns=['directions', 'directions_clean'])
df2

,rt,rtnm,rtclr,rtdd,dir_id,dir_nm
0,1,Bronzeville/Union Station,#336633,1,Northbound,Northbound
0,1,Bronzeville/Union Station,#336633,1,Southbound,Southbound
1,2,Hyde Park Express,#993366,2,Northbound,Northbound
1,2,Hyde Park Express,#993366,2,Southbound,Southbound
2,3,King Drive,#009900,3,Northbound,Northbound
...,...,...,...,...,...,...
121,192,U. of Chicago Hospitals Express,#cc3300,192,Southbound,Southbound
122,201,Central/Ridge,#996633,201,Eastbound,Eastbound
122,201,Central/Ridge,#996633,201,Westbound,Westbound
123,206,Evanston Circulator,#666600,206,Eastbound,Eastbound


## GET Directions

In [8]:
tracker.getdirections(rt='8')

{'bustime-response': {'directions': [{'id': 'Northbound',
    'name': 'Northbound'},
   {'id': 'Southbound', 'name': 'Southbound'}]}}

## GET Stops

In [27]:
df2['stops'] = df2.progress_apply(lambda x: tracker.getstops(rt=x.rt, dir=x.dir_id), axis=1)
df2

100%|██████████| 247/247 [01:54<00:00,  2.16it/s]


,rt,rtnm,rtclr,rtdd,dir_id,dir_nm,stops
0,1,Bronzeville/Union Station,#336633,1,Northbound,Northbound,{'bustime-response': {'stops': [{'stpid': '156...
0,1,Bronzeville/Union Station,#336633,1,Southbound,Southbound,{'bustime-response': {'stops': [{'stpid': '160...
1,2,Hyde Park Express,#993366,2,Northbound,Northbound,{'bustime-response': {'stops': [{'stpid': '170...
1,2,Hyde Park Express,#993366,2,Southbound,Southbound,{'bustime-response': {'stops': [{'stpid': '487...
2,3,King Drive,#009900,3,Northbound,Northbound,{'bustime-response': {'stops': [{'stpid': '218...
...,...,...,...,...,...,...,...
121,192,U. of Chicago Hospitals Express,#cc3300,192,Southbound,Southbound,{'bustime-response': {'stops': [{'stpid': '140...
122,201,Central/Ridge,#996633,201,Eastbound,Eastbound,{'bustime-response': {'stops': [{'stpid': '154...
122,201,Central/Ridge,#996633,201,Westbound,Westbound,{'bustime-response': {'stops': [{'stpid': '149...
123,206,Evanston Circulator,#666600,206,Eastbound,Eastbound,{'bustime-response': {'stops': [{'stpid': '143...


In [32]:
df2['stops_cln'] = df2['stops'].apply(lambda x: x['bustime-response']['stops'])
df_stops = df2.explode('stops_cln')
df_stops['stpid'] = df_stops['stops_cln'].apply(lambda x: x['stpid'])
df_stops['stpnm'] = df_stops['stops_cln'].apply(lambda x: x['stpnm'])
df_stops['lat'] = df_stops['stops_cln'].apply(lambda x: x['lat'])
df_stops['lon'] = df_stops['stops_cln'].apply(lambda x: x['lon'])
df3 = df_stops.drop(columns=['stops', 'stops_cln'])
df3

,rt,rtnm,rtclr,rtdd,dir_id,dir_nm,stpid,stpnm,lat,lon
0,1,Bronzeville/Union Station,#336633,1,Northbound,Northbound,1564,3000 S Michigan,41.840607,-87.623207
0,1,Bronzeville/Union Station,#336633,1,Northbound,Northbound,17046,Adams & Clark,41.879457,-87.631083
0,1,Bronzeville/Union Station,#336633,1,Northbound,Northbound,1108,Adams & S. Wacker,41.879177,-87.636101
0,1,Bronzeville/Union Station,#336633,1,Northbound,Northbound,4732,Adams & State,41.879491,-87.628093
0,1,Bronzeville/Union Station,#336633,1,Northbound,Northbound,77,Adams & Wabash,41.879580,-87.625973
...,...,...,...,...,...,...,...,...,...,...
123,206,Evanston Circulator,#666600,206,Westbound,Westbound,14258,Ridge & Monroe,42.030722,-87.685912
123,206,Evanston Circulator,#666600,206,Westbound,Westbound,18322,Ridge & Mulford,42.023166,-87.685016
123,206,Evanston Circulator,#666600,206,Westbound,Westbound,14254,Ridge & Oakton,42.026477,-87.685355
123,206,Evanston Circulator,#666600,206,Westbound,Westbound,14256,Ridge & Seward,42.028933,-87.685657


In [9]:
tracker.getstops(rt='8', dir='Southbound')

{'bustime-response': {'stops': [{'stpid': '5776',
    'stpnm': '1700 N Halsted',
    'lat': 41.912085,
    'lon': -87.648352000001},
   {'stpid': '5774',
    'stpnm': '1900 N Halsted',
    'lat': 41.916024999999,
    'lon': -87.648524},
   {'stpid': '5812',
    'stpnm': '2300 S Halsted ',
    'lat': 41.850904999999,
    'lon': -87.646533},
   {'stpid': '5848',
    'stpnm': '6400 S Halsted',
    'lat': 41.778039000001,
    'lon': -87.644870999999},
   {'stpid': '5804',
    'stpnm': 'Halsted & 14th Place',
    'lat': 41.862536000001,
    'lon': -87.646866999998},
   {'stpid': '5806',
    'stpnm': 'Halsted & 16th Street',
    'lat': 41.859465000001,
    'lon': -87.646773000001},
   {'stpid': '17904',
    'stpnm': 'Halsted & 18th Street',
    'lat': 41.857250000001,
    'lon': -87.646714000002},
   {'stpid': '17905',
    'stpnm': 'Halsted & 26th Street',
    'lat': 41.845006,
    'lon': -87.646532000001},
   {'stpid': '5815',
    'stpnm': 'Halsted & 28th Street',
    'lat': 41.843603999999

## GET Bustime Predictions

In [ ]:
tracker.getpredictions(stpid='5776')

## GET Vehicles

In [25]:
tracker.getvehicles(rt='8')

{'bustime-response': {'vehicle': [{'vid': '1209',
    'tmstmp': '20240228 22:55:36',
    'lat': '41.941782448508526',
    'lon': '-87.64932250976562',
    'hdg': '178',
    'pid': 9368,
    'rt': '8',
    'des': '79th',
    'pdist': 3264,
    'dly': False,
    'tatripid': '1078429',
    'origtatripno': '251983095',
    'tablockid': '8 -708',
    'zone': '',
    'mode': 1,
    'psgld': 'N/A',
    'stst': 82320,
    'stsd': '2024-02-28'},
   {'vid': '1327',
    'tmstmp': '20240228 22:55:48',
    'lat': '41.87947047840465',
    'lon': '-87.64727783203125',
    'hdg': '178',
    'pid': 9368,
    'rt': '8',
    'des': '79th',
    'pdist': 26016,
    'dly': False,
    'tatripid': '1078428',
    'origtatripno': '251983149',
    'tablockid': '8 -705',
    'zone': '',
    'mode': 1,
    'psgld': 'N/A',
    'stst': 81210,
    'stsd': '2024-02-28'},
   {'vid': '7932',
    'tmstmp': '20240228 22:55:46',
    'lat': '41.85533696954901',
    'lon': '-87.64656829833984',
    'hdg': '179',
    'pid': 9

In [57]:
tracker.getvehicles(vid='7991')

{'bustime-response': {'vehicle': [{'vid': '7991',
    'tmstmp': '20240228 23:26:50',
    'lat': '41.87372272572619',
    'lon': '-87.62431335449219',
    'hdg': '179',
    'pid': 18415,
    'rt': '3',
    'des': '95th Red Line',
    'pdist': 10910,
    'dly': False,
    'tatripid': '1080329',
    'origtatripno': '251981013',
    'tablockid': 'N4  -793',
    'zone': '',
    'mode': 1,
    'psgld': 'N/A',
    'stst': -3000,
    'stsd': '2024-02-29'}]}}

## GET Patterns (road segments)

In [ ]:
tracker.getpatterns(rt='8')

In [35]:
df_segs = df.drop(columns=['directions', 'directions_clean'])
df_segs['pattern'] = df_segs['rt'].progress_apply(lambda x: tracker.getpatterns(rt=x))
df_segs

100%|██████████| 124/124 [01:01<00:00,  2.01it/s]


,rt,rtnm,rtclr,rtdd,pattern
0,1,Bronzeville/Union Station,#336633,1,"{'bustime-response': {'ptr': [{'pid': 6351, 'l..."
1,2,Hyde Park Express,#993366,2,"{'bustime-response': {'ptr': [{'pid': 5528, 'l..."
2,3,King Drive,#009900,3,"{'bustime-response': {'ptr': [{'pid': 18414, '..."
3,4,Cottage Grove,#cc3300,4,"{'bustime-response': {'ptr': [{'pid': 19380, '..."
4,X4,Cottage Grove Express,#006666,X4,"{'bustime-response': {'ptr': [{'pid': 19372, '..."
...,...,...,...,...,...
119,171,U. of Chicago/Hyde Park,#336633,171,"{'bustime-response': {'ptr': [{'pid': 14108, '..."
120,172,U. of Chicago/Kenwood,#993366,172,"{'bustime-response': {'ptr': [{'pid': 14103, '..."
121,192,U. of Chicago Hospitals Express,#cc3300,192,"{'bustime-response': {'ptr': [{'pid': 2671, 'l..."
122,201,Central/Ridge,#996633,201,"{'bustime-response': {'ptr': [{'pid': 4318, 'l..."


In [39]:
df_segs['pattern_clean'] = df_segs['pattern'].apply(lambda x: x['bustime-response']['ptr'])
df4 = df_segs.explode('pattern_clean').drop(columns=['pattern'])
df4

,rt,rtnm,rtclr,rtdd,pattern_clean
0,1,Bronzeville/Union Station,#336633,1,"{'pid': 6351, 'ln': 23541.0, 'rtdir': 'Southbo..."
0,1,Bronzeville/Union Station,#336633,1,"{'pid': 8085, 'ln': 25038.0, 'rtdir': 'Northbo..."
1,2,Hyde Park Express,#993366,2,"{'pid': 5528, 'ln': 54663.0, 'rtdir': 'Southbo..."
1,2,Hyde Park Express,#993366,2,"{'pid': 5529, 'ln': 57979.0, 'rtdir': 'Southbo..."
1,2,Hyde Park Express,#993366,2,"{'pid': 5530, 'ln': 55196.0, 'rtdir': 'Northbo..."
...,...,...,...,...,...
121,192,U. of Chicago Hospitals Express,#cc3300,192,"{'pid': 2671, 'ln': 45819.0, 'rtdir': 'Northbo..."
122,201,Central/Ridge,#996633,201,"{'pid': 4318, 'ln': 45646.0, 'rtdir': 'Eastbou..."
122,201,Central/Ridge,#996633,201,"{'pid': 4319, 'ln': 45471.0, 'rtdir': 'Westbou..."
123,206,Evanston Circulator,#666600,206,"{'pid': 3190, 'ln': 32023.0, 'rtdir': 'Eastbou..."


In [44]:
df4['pid'] = df4['pattern_clean'].apply(lambda x: x['pid'])
df4['ln'] = df4['pattern_clean'].apply(lambda x: x['ln'])
df4['rtdir'] = df4['pattern_clean'].apply(lambda x: x['rtdir'])
df4['pt'] = df4['pattern_clean'].apply(lambda x: x['pt'])
df5 = df4.explode('pt').drop(columns=['pattern_clean'])
df5

,rt,rtnm,rtclr,rtdd,pid,ln,rtdir,pt
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,"{'seq': 1, 'lat': 41.874499999999, 'lon': -87...."
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,"{'seq': 2, 'lat': 41.8745, 'lon': -87.64432, '..."
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,"{'seq': 3, 'lat': 41.87436, 'lon': -87.64432, ..."
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,"{'seq': 4, 'lat': 41.87438, 'lon': -87.64242, ..."
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,"{'seq': 5, 'lat': 41.87466, 'lon': -87.64243, ..."
...,...,...,...,...,...,...,...,...
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,"{'seq': 160, 'lat': 42.06459, 'lon': -87.72576..."
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,"{'seq': 161, 'lat': 42.064528000001, 'lon': -8..."
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,"{'seq': 162, 'lat': 42.06459, 'lon': -87.72576..."
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,"{'seq': 163, 'lat': 42.06457, 'lon': -87.72398..."


In [49]:
df5['pt'].iloc[0]

{'seq': 1,
 'lat': 41.874499999999,
 'lon': -87.644385999998,
 'typ': 'S',
 'stpid': '13155',
 'stpnm': 'Desplaines & Harrison Terminal',
 'pdist': 0.0}

In [50]:
df5['pt'].iloc[1]

{'seq': 2, 'lat': 41.8745, 'lon': -87.64432, 'typ': 'W', 'pdist': 0.0}

In [53]:
df5['seq'] = df5['pt'].apply(lambda x: x['seq'])
df5['lat'] = df5['pt'].apply(lambda x: x['lat'])
df5['lon'] = df5['pt'].apply(lambda x: x['lon'])
df5['typ'] = df5['pt'].apply(lambda x: x['typ'])
df5['stpid'] = df5['pt'].apply(lambda x: x.get('stpid'))
df5['stpnm'] = df5['pt'].apply(lambda x: x.get('stpnm'))
df5['pdist'] = df5['pt'].apply(lambda x: x['pdist'])
df6 = df5.drop(columns=['pt'])
df6

,rt,rtnm,rtclr,rtdd,pid,ln,rtdir,seq,lat,lon,typ,stpid,stpnm,pdist
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,1,41.874500,-87.644386,S,13155,Desplaines & Harrison Terminal,0.0
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,2,41.874500,-87.644320,W,None,None,0.0
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,3,41.874360,-87.644320,W,None,None,0.0
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,4,41.874380,-87.642420,W,None,None,0.0
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,5,41.874660,-87.642430,W,None,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,160,42.064590,-87.725760,W,None,None,0.0
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,161,42.064528,-87.725758,S,14686,Central Street & Prospect,33116.0
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,162,42.064590,-87.725760,W,None,None,0.0
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,163,42.064570,-87.723988,W,None,None,0.0


In [67]:
df6

,rt,rtnm,rtclr,rtdd,pid,ln,rtdir,seq,lat,lon,typ,stpid,stpnm,pdist
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,1,41.874500,-87.644386,S,13155,Desplaines & Harrison Terminal,0.0
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,2,41.874500,-87.644320,W,None,None,0.0
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,3,41.874360,-87.644320,W,None,None,0.0
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,4,41.874380,-87.642420,W,None,None,0.0
0,1,Bronzeville/Union Station,#336633,1,6351,23541.0,Southbound,5,41.874660,-87.642430,W,None,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,160,42.064590,-87.725760,W,None,None,0.0
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,161,42.064528,-87.725758,S,14686,Central Street & Prospect,33116.0
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,162,42.064590,-87.725760,W,None,None,0.0
123,206,Evanston Circulator,#666600,206,8117,33631.0,Westbound,163,42.064570,-87.723988,W,None,None,0.0


In [58]:
df.drop(columns=['directions', 'directions_clean']).to_csv('routes.csv', index=False)
df2.drop(columns=['stops', 'stops_cln']).to_csv('routes-w-directions.csv', index=False)
df3.to_csv('stops-by-route-and-direction.csv', index=False)
df5.drop(columns=['pt']).to_csv('patterns-by-route-and-direction.csv', index=False)

,rt,rtnm,rtclr,rtdd,directions,directions_clean
0,1,Bronzeville/Union Station,#336633,1,{'bustime-response': {'directions': [{'id': 'N...,"[{'id': 'Northbound', 'name': 'Northbound'}, {..."
1,2,Hyde Park Express,#993366,2,{'bustime-response': {'directions': [{'id': 'N...,"[{'id': 'Northbound', 'name': 'Northbound'}, {..."
2,3,King Drive,#009900,3,{'bustime-response': {'directions': [{'id': 'N...,"[{'id': 'Northbound', 'name': 'Northbound'}, {..."
3,4,Cottage Grove,#cc3300,4,{'bustime-response': {'directions': [{'id': 'N...,"[{'id': 'Northbound', 'name': 'Northbound'}, {..."
4,X4,Cottage Grove Express,#006666,X4,{'bustime-response': {'directions': [{'id': 'N...,"[{'id': 'Northbound', 'name': 'Northbound'}, {..."
...,...,...,...,...,...,...
119,171,U. of Chicago/Hyde Park,#336633,171,{'bustime-response': {'directions': [{'id': 'N...,"[{'id': 'Northbound', 'name': 'Northbound'}, {..."
120,172,U. of Chicago/Kenwood,#993366,172,{'bustime-response': {'directions': [{'id': 'N...,"[{'id': 'Northbound', 'name': 'Northbound'}, {..."
121,192,U. of Chicago Hospitals Express,#cc3300,192,{'bustime-response': {'directions': [{'id': 'N...,"[{'id': 'Northbound', 'name': 'Northbound'}, {..."
122,201,Central/Ridge,#996633,201,{'bustime-response': {'directions': [{'id': 'E...,"[{'id': 'Eastbound', 'name': 'Eastbound'}, {'i..."


## GET Service Bulletins  

In [54]:
tracker.getservicebulletins(rt='8')

{'bustime-response': {'sb': [{'nm': '8 Alerts',
    'sbj': 'Check for #8 alerts',
    'dtl': 'For alerts, maps and detailed route information, visit <a href="https://transitchicago.com/bus/8">#8 Halsted Route Information</a> on our website. You can also sign up to have alerts sent by text or e-mail by signing up for <a href="https://transitchicago.com/updates">CTA Updates</a>.',
    'brf': 'For alerts and detailed #8 bus route info, visit https://transitchicago.com/bus/8 - get alerts sent to you via transitchicago.com/updates',
    'prty': 'Low',
    'srvc': [{'rt': '8', 'rtdir': '', 'stpid': '', 'stpnm': ''}],
    'mod': '',
    'url': ''}]}}

## GET Detours

In [55]:
tracker.getdetours(rt='8')

{'bustime-response': {}}

## GET Locale List

In [56]:
tracker.getlocalelist()

{'bustime-response': {'locale': [{'localestring': 'en',
    'displayname': 'English'},
   {'localestring': 'es', 'displayname': 'Spanish'}]}}